In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import pandas as pd
import numpy as np
!pip install emoji
import nltk
import re
nltk.download('punkt')
from nltk.corpus import stopwords
import emoji
nltk.download('stopwords')
from collections import Counter
from nltk.tokenize import word_tokenize
!pip install fasttext
import fasttext
import ast
import json
import fasttext.util

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
data = pd.read_csv("/content/drive/MyDrive/Multiclass_Classification_Internship_NIT_Patna/contextual_combined_data.csv")

In [5]:
data

,tweet_id,text,contextual_label
0,1214794148504457217,It’s not a fight for dharma it’s embracing it....,NONE
1,1242560043494621185,bhenchod mulk ki maa chud rahi hai aur tum log...,SHOF
2,1335539265015517186,#babrimasjid\n#BabriMasjid was and always will...,NONE
3,1467692528308686850,#babrimasjid\n#BabriMasjid was and always will...,SHOF
4,1467694052644511745,#babrimasjid\n#BabriMasjid was and always will...,CHOF
...,...,...,...
4909,1534729188842033152,One woman 57 muslim country 😂😂 #QatarExposed h...,NONE
4910,1534750107673010176,One woman 57 muslim country 😂😂 #QatarExposed h...,SHOF
4911,1534754537914806272,One woman 57 muslim country 😂😂 #QatarExposed h...,SHOF
4912,1534757926853783552,One woman 57 muslim country 😂😂 #QatarExposed h...,SHOF


In [8]:
data['contextual_label'].value_counts()

,count
contextual_label,
NONE,2390
SHOF,1636
CHOF,888


In [9]:
def load_and_preprocess_data(file_path):
    # Load the data
    df = pd.read_csv(file_path)
    print(df)
    print(df['contextual_label'].value_counts())

    return df

In [10]:
file_path= r'/content/drive/MyDrive/Multiclass_Classification_Internship_NIT_Patna/contextual_combined_data.csv'

In [11]:
df = load_and_preprocess_data(file_path)

                 tweet_id                                               text  \
0     1214794148504457217  It’s not a fight for dharma it’s embracing it....   
1     1242560043494621185  bhenchod mulk ki maa chud rahi hai aur tum log...   
2     1335539265015517186  #babrimasjid\n#BabriMasjid was and always will...   
3     1467692528308686850  #babrimasjid\n#BabriMasjid was and always will...   
4     1467694052644511745  #babrimasjid\n#BabriMasjid was and always will...   
...                   ...                                                ...   
4909  1534729188842033152  One woman 57 muslim country 😂😂 #QatarExposed h...   
4910  1534750107673010176  One woman 57 muslim country 😂😂 #QatarExposed h...   
4911  1534754537914806272  One woman 57 muslim country 😂😂 #QatarExposed h...   
4912  1534757926853783552  One woman 57 muslim country 😂😂 #QatarExposed h...   
4913  1534722278768136192  One woman 57 muslim country 😂😂 #QatarExposed h...   

     contextual_label  
0              

In [12]:
def clean_text(text):
    # Convert emojis to text
#     text = emoji.demojize(text, delimiters=("", ""))
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F1E0-\U0001F1FF"  # flags (iOS)
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE
    )
    text = emoji_pattern.sub(r'', text)

    # Remove URLs
    text = re.sub(r'http\S+', '', text)

    # Remove digits
    text = re.sub(r'\d+', '', text)

    # Remove special characters except for spaces and Hindi characters
    text = re.sub(r'[^\w\s\u0900-\u097F]', '', text)  # Keep Hindi characters and spaces

    # Normalize multiple spaces to a single space
    text = re.sub(r'\s+', ' ', text).strip()

    return text


def load_data(file_path):
    df = pd.read_csv(file_path)

    # Apply text cleaning
    df['cleaned_text'] = df['text'].apply(clean_text)

    # Drop the original 'text' column if no longer needed
    df.drop(['text'], axis=1, inplace=True)

    return df
# Load and preprocess the data
data = load_data(file_path)
print(df.head())

              tweet_id                                               text  \
0  1214794148504457217  It’s not a fight for dharma it’s embracing it....   
1  1242560043494621185  bhenchod mulk ki maa chud rahi hai aur tum log...   
2  1335539265015517186  #babrimasjid\n#BabriMasjid was and always will...   
3  1467692528308686850  #babrimasjid\n#BabriMasjid was and always will...   
4  1467694052644511745  #babrimasjid\n#BabriMasjid was and always will...   

  contextual_label  
0             NONE  
1             SHOF  
2             NONE  
3             SHOF  
4             CHOF  


In [13]:
df['cleaned_text'] = df['text'].apply(lambda text: clean_text(text) if pd.notnull(text) else text)

In [14]:
nltk_stopwords = set(stopwords.words('english'))

In [15]:
text_column = 'text'

In [16]:
stop_words = set(stopwords.words('english'))

In [17]:
def extract_stopwords(text):
    words = re.findall(r'\b\w+\b', text.lower())
    stopwords_in_text = [word for word in words if word in stop_words]
    return stopwords_in_text

In [18]:
all_stopwords = []
for text in df[text_column].dropna():
    all_stopwords.extend(extract_stopwords(text))

In [19]:
vocabulary_of_stopwords = set(all_stopwords)

In [20]:
print("Vocabulary of stopwords:")
print(vocabulary_of_stopwords)

Vocabulary of stopwords:
{'t', 'few', 'about', 'couldn', 'now', 'then', 'my', 'until', 'themselves', 'be', 'through', 'off', 'shouldn', 'doesn', 're', 'have', 'hadn', 'he', 'or', 'when', 'under', 'haven', 'its', 'was', 'these', 'were', 'yourselves', 'ain', 'below', 'isn', 'are', 'more', 'we', 'our', 'i', 'am', 'weren', 'once', 'during', 'she', 'had', 'than', 'yourself', 'which', 'does', 'for', 'all', 'if', 'how', 'a', 'theirs', 'no', 'same', 'by', 'hasn', 'myself', 'himself', 'who', 'from', 'being', 'her', 'herself', 'wouldn', 'of', 'over', 'having', 'has', 'in', 'me', 'most', 'further', 'but', 'such', 'an', 'other', 'they', 'them', 'ours', 'what', 'their', 'won', 'did', 'd', 'the', 'didn', 'again', 'ourselves', 'do', 'each', 'aren', 'been', 'should', 'you', 'so', 'very', 'itself', 'just', 'after', 'own', 'not', 'to', 'why', 'at', 'wasn', 'ma', 'on', 'him', 'down', 'his', 'between', 'up', 'above', 'those', 'with', 'where', 'can', 'as', 'hers', 'any', 'only', 'because', 'some', 'too', '

In [21]:
stopword_counts = Counter(all_stopwords)
print("\nMost common stopwords:")
print(stopword_counts.most_common())


Most common stopwords:
[('the', 6927), ('to', 4581), ('t', 4180), ('in', 3815), ('and', 3795), ('is', 3793), ('of', 3763), ('a', 3229), ('for', 2842), ('s', 2293), ('you', 2216), ('was', 1809), ('on', 1761), ('it', 1730), ('this', 1520), ('by', 1454), ('be', 1346), ('are', 1308), ('that', 1304), ('who', 1260), ('not', 1228), ('have', 1144), ('i', 1090), ('if', 1084), ('no', 1076), ('from', 1002), ('they', 973), ('will', 969), ('she', 890), ('with', 855), ('can', 834), ('we', 814), ('as', 794), ('what', 775), ('at', 667), ('so', 666), ('her', 655), ('has', 648), ('he', 637), ('its', 636), ('your', 619), ('but', 601), ('when', 594), ('their', 578), ('such', 549), ('against', 541), ('why', 539), ('just', 526), ('own', 520), ('an', 480), ('about', 476), ('itself', 470), ('or', 469), ('there', 464), ('all', 446), ('were', 438), ('his', 421), ('do', 412), ('me', 405), ('any', 384), ('don', 360), ('only', 348), ('them', 348), ('how', 337), ('him', 325), ('after', 319), ('here', 315), ('more'

In [22]:
def remove_english_stopwords(text):
    words = nltk.word_tokenize(text.lower())
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

In [23]:
# Apply stopword removal to the dataset
df['cleaned_text'] = df['cleaned_text'].apply(lambda text: remove_english_stopwords(text) if pd.notnull(text) else text)

In [24]:
hindi_stopwords = [
    "अंदर", "अगर", "अत", "अथवा", "अदि", "अधिक", "अधिकांश", "अन्य", "अप", "अपना", "अपनि", "अपनी", "अपने", "अब", "अभि", "अभी", "आगे", "आज", "आदि", "आप", "आपका",
    "इंहिं", "इंहें", "इंहों", "इतयादि", "इत्यादि", "इन", "इनका", "इनकी", "इनके", "इन्हीं", "इन्हें", "इन्हों", "इस", "इसका", "इसकि", "इसकी", "इसके", "इसमें",
    "इसलिए", "इसि", "इसी", "इसे", "उंहिं", "उंहें", "उंहों", "उन", "उनका", "उनकि", "उनकी", "उनके", "उनको", "उन्हीं", "उन्हें", "उन्हों", "उन्होने", "उस", "उसका",
    "उसकी", "उसके", "उसि", "उसी", "उसे", "ऊपर", "एक", "एवं", "एस", "एसे", "ऐसा", "ऐसे", "ओर", "और", "कइ", "कई", "कब", "कभी", "कम", "कर", "करता", "करते",
    "करना", "करने", "करें", "कल", "कहते", "कहा", "कहाँ", "का", "काफि", "काफ़ी", "कि", "किंहें", "किंहों", "किए", "कितना", "किन्हें", "किन्हों", "किया", "किर",
    "किस", "किसको", "किसि", "किसी", "किसे", "की", "कुछ", "कुल", "के", "कैसे", "को", "कोइ", "कोई", "कोन", "कोनसा", "कौन", "कौनसा", "क्या", "क्यूंकि", "क्यों",
    "खिलाफ", "खुद", "गई", "गए", "गया", "गयी", "गये", "घर", "चाहिए", "जब", "जबकि", "जरा", "जहाँ", "जहां", "जा", "जाता", "जाती", "जाते", "जाने", "जायेंगे",
    "जिंहें", "जिंहों", "जितना", "जिधर", "जिन", "जिन्हें", "जिन्हों", "जिस", "जिसमें", "जिससे", "जिसे", "जीधर", "जेसा", "जेसे", "जैसा", "जैसे", "जो", "ज्यादा",
    "तक", "तथा", "तब", "तरह", "तहत", "तिंहें", "तिंहों", "तिन", "तिन्हें", "तिन्हों", "तिस", "तिसे", "तुम", "तुम्हारा", "तो", "था", "थि", "थी", "थे", "दबारा",
    "दवारा", "दिया", "दुबारा", "दुसरा", "दुसरे", "दूर", "दूसरा", "दूसरे", "दो", "दोनो", "दोनों", "दौरान", "द्वारा", "न", "नहिं", "नहीं", "ना", "निकल", "निचे",
    "निहायत", "नीचे", "ने", "पडा", "पर", "परंतु", "पहले", "पुरा", "पूरा", "पूरे", "पे", "प्रति", "प्रत्येक", "फिर", "बंद", "बड़ा", "बड़े", "बनि", "बनी", "बहि",
    "बही", "बहुत", "बाद", "बार", "बारे", "बाला", "बाहर", "बिलकुल", "बीच", "भि", "भितर", "भी", "भीतर", "मगर", "मात्र", "माध्यम", "मानो", "मुझको", "मे", "में",
    "मेरा", "मै", "मैं", "यदि", "यह", "यहाँ", "यहां", "यहि", "यही", "या", "यिह", "ये", "रखें", "रवासा", "रहती", "रहा", "रही", "रहे", "ऱ्वासा", "लिए", "लिया",
    "लिये", "लेकर", "लेकिन", "व", "वगेरह", "वरग", "वर्ग", "वह", "वहाँ", "वहां", "वहिं", "वहीं", "वाले", "वुह", "वे", "वग़ैरह", "संग", "सकता", "सकती", "सकते",
    "सबसे", "सभि", "सभी", "समय", "समान", "साथ", "साबुत", "साभ", "सारा", "सारे", "से", "सो", "स्थान", "स्वयं", "हम", "हमने", "हमारा", "हि", "ही", "हुअ", "हुआ",
    "हुइ", "हुई", "हुए", "हुये", "हूँ", "हे", "हें", "है", "हैं", "हो", "होता", "होति", "होती", "होते", "होना", "होने", "के"
]

In [25]:
# Ensure the column with text data is correctly specified (replace 'text_column' with the actual column name)
text_column = 'text'  # Replace with the correct column name containing text data

# Tokenize and preprocess the text data
tokens = [word_tokenize(doc.lower()) for doc in df[text_column].dropna()]  # Tokenize and convert to lowercase

# Flatten the list of tokens
all_tokens = [token for sublist in tokens for token in sublist]

# Define a function to check if a word is in Hindi
def is_hindi(word):
    # Check if the word contains Devanagari characters
    return bool(re.search(r'[\u0900-\u097F]', word))

# Get the vocabulary of only Hindi words
hindi_vocabulary = {token for token in all_tokens if is_hindi(token)}

print("Hindi Vocabulary:", hindi_vocabulary)
print("Hindi Vocabulary size:", len(hindi_vocabulary))

Hindi Vocabulary: {'खूबियों', 'चूहे', 'दिखेगा', 'कांटे', 'टैक्स', 'किसने', 'नोचना', 'धार्मिक', 'मुफ्त', 'मरेंगे', 'बंदा।', 'उसकी', 'जबरदस्ती', 'सूची', 'है👌👌', 'लिखने', 'मटेरियल', 'चापलूसों', 'चुस्लाम', 'सार्वजानिकतौर', 'व्यक्ति', 'सर्वोपरि', 'झूठी', 'मां', 'संघी', 'पूजती', 'पंड़ित', 'पढों\u200c।', 'दिया।', 'जबरदस्त', 'रैली', 'करवायेंगे', 'रहेगा।', 'हथियार', 'अजीब', 'सोनिया', 'राजपाठ', 'नही😭', 'विनाश', 'झन्नाटेदार', 'अमृत', 'पापिन', 'गई', 'भगवा', 'मूर्खो', 'सिक्ख', 'फेमस', 'मूत्र', "'आतंकी", 'चलेगी', 'दादी', 'देहांत', 'चवन्नी', 'होता।उसको', 'बनाया', 'टमाटर', 'सपूत', 'हटाया', 'चमकाने', 'मैगज़ीन', 'उंगलियों', 'निकाला', 'बहूत', 'पाक', 'दीदी', 'तैयारी', 'किरदारों', 'कमाया', 'भूलेंगे', 'जब', 'ऐरे', 'भेजे।', 'लेगी', 'सच्ची', 'सेलिब्रिटी', 'हिस्से', 'हिंसक', 'सिर्फ़', 'इतिहास', 'इलाकों', 'प्रकाश', 'जातीय', 'बाजे', 'मामला', 'सादगी', 'दलित', 'तडीपार', 'थे', 'है😊😊', 'किताब', 'हल्के', 'हरामखोर', 'बेवकूफ', 'ट्रांसजेंडर', 'जिदगी', 'आपने।', 'चाटो', 'वाही', 'उन्होंने', 'जागो', 'हूं', 'उगलते', 'दिहो।',

In [26]:
def remove_hindi_stopwords(text):
    words = word_tokenize(text.lower())  # Tokenize the text into words
    filtered_words = [word for word in words if word not in hindi_stopwords]  # Remove Hindi stopwords
    return ' '.join(filtered_words)

In [27]:
df['cleaned_text'] = df['cleaned_text'].apply(lambda text: remove_hindi_stopwords(text) if pd.notnull(text) else text)

In [28]:
output_file_path = r'/content/drive/MyDrive/Multiclass_Classification_Internship_NIT_Patna/multiclass_preprocessed_data.csv'

# Save the cleaned data to a CSV file
df.to_csv(output_file_path, index=False)

print(f"Preprocessed data saved to {output_file_path}")

Preprocessed data saved to /content/drive/MyDrive/Multiclass_Classification_Internship_NIT_Patna/multiclass_preprocessed_data.csv


In [29]:
# Load the CSV file
file_path = r'/content/drive/MyDrive/Multiclass_Classification_Internship_NIT_Patna/multiclass_preprocessed_data.csv'
df = pd.read_csv(file_path)

# Function to tokenize text
def tokenize_text(text):
    return text.split(" ")

# Function to remove poorna virams from within tokens
def remove_purna_viram(tokens):
    return [token.replace('।', '') for token in tokens]

# Function to remove Urdu text
def remove_urdu_text(tokens):
    urdu_range = ('\u0600', '\u06FF')  # Unicode range for Arabic script
    return [token for token in tokens if not any(urdu_range[0] <= char <= urdu_range[1] for char in token)]

# Function to remove underscores
def remove_underscores(tokens):
    return [token.replace('_', '') for token in tokens]

# Function to remove empty tokens
def remove_empty_tokens(tokens):
    return [token for token in tokens if token]

# Function to check if a token contains both Hindi and non-Hindi characters
def contains_hindi_and_non_hindi(token):
    hindi_pattern = re.compile('[\u0900-\u097F]')
    non_hindi_pattern = re.compile('[^\u0900-\u097F\s]')
    contains_hindi = bool(hindi_pattern.search(token))
    contains_non_hindi = bool(non_hindi_pattern.search(token))
    return contains_hindi and contains_non_hindi

# Function to split tokens containing both Hindi and non-Hindi characters
def split_hindi_and_non_hindi(token):
    hindi_pattern = re.compile('([\u0900-\u097F]+)')
    non_hindi_pattern = re.compile('([^\u0900-\u097F]+)')
    split_tokens = hindi_pattern.split(token)
    split_tokens = [sub_token for token in split_tokens for sub_token in non_hindi_pattern.split(token) if sub_token]
    return split_tokens

# Apply the function to split the tokens and update the tokenized_text
def process_tokenized_text(tokenized_text):
    processed_tokens = []
    for token in tokenized_text:
        if contains_hindi_and_non_hindi(token):
            processed_tokens.extend(split_hindi_and_non_hindi(token))
        else:
            processed_tokens.append(token)
    return processed_tokens

# Function to process the dataframe
def tokenize_dataframe(df, text_column='cleaned_text'):
    df['tokenized_text'] = df[text_column].apply(lambda text: tokenize_text(text) if pd.notnull(text) else [])
    df['tokenized_text'] = df['tokenized_text'].apply(remove_purna_viram)
    df['tokenized_text'] = df['tokenized_text'].apply(remove_urdu_text)
    df['tokenized_text'] = df['tokenized_text'].apply(remove_underscores)
    df['tokenized_text'] = df['tokenized_text'].apply(remove_empty_tokens)
    df['tokenized_text'] = df['tokenized_text'].apply(process_tokenized_text)
    return df

# Process the dataframe
df = tokenize_dataframe(df, text_column='cleaned_text')

# Save the processed DataFrame to a CSV file
output_file_path = r'/content/drive/MyDrive/Multiclass_Classification_Internship_NIT_Patna/multiclass_tokenized_cleaned_data.csv'
df.to_csv(output_file_path, index=False)

# Display the first few rows of the dataframe
print(df.head())

              tweet_id                                               text  \
0  1214794148504457217  It’s not a fight for dharma it’s embracing it....   
1  1242560043494621185  bhenchod mulk ki maa chud rahi hai aur tum log...   
2  1335539265015517186  #babrimasjid\n#BabriMasjid was and always will...   
3  1467692528308686850  #babrimasjid\n#BabriMasjid was and always will...   
4  1467694052644511745  #babrimasjid\n#BabriMasjid was and always will...   

  contextual_label                                       cleaned_text  \
0             NONE       fight dharma embracing hindusagainsthindutva   
1             SHOF  bhenchod mulk ki maa chud rahi hai aur tum log...   
2             NONE  babrimasjid babrimasjid always mosquerecently ...   
3             SHOF  babrimasjid babrimasjid always mosquerecently ...   
4             CHOF  babrimasjid babrimasjid always mosquerecently ...   

                                      tokenized_text  
0  [fight, dharma, embracing, hindusagainst

In [31]:
# Load the CSV file into a DataFrame
df = pd.read_csv('/content/drive/MyDrive/Multiclass_Classification_Internship_NIT_Patna/multiclass_tokenized_cleaned_data.csv')

# Specify the column you want to delete
column_to_delete = 'text'

# Delete the column
df.drop(columns=['text'], inplace=True)

# Save the updated DataFrame back to the same CSV file, effectively overwriting it
df.to_csv('tokenized_cleaned_data.csv', index=False)

# Specify the column you want to delete
column_to_delete = 'cleaned_text'

# Delete the column
df.drop(columns=['cleaned_text'], inplace=True)

# Save the updated DataFrame back to the same CSV file, effectively overwriting it
df.to_csv('tokenized_text.csv', index=False)

In [32]:
# from Preprocessed_and_tokenized import tokenize_dataframe

def load_and_process_data(file_path):
    """Load and process the data."""
    # Load the CSV file
    df = pd.read_csv(file_path)

    # Process the dataframe
    df = tokenize_dataframe(df, text_column='cleaned_text')
    return df

def split_tokens_by_language(df, tokenized_text_column='tokenized_text'):
    """Split tokens into Hindi and English sets."""
    hindi_tokens = set()
    english_tokens = set()

    for tokens in df[tokenized_text_column]:
        for token in tokens:
            if re.search('[\u0900-\u097F]', token):  # Check for Hindi characters
                hindi_tokens.add(token)
            else:
                english_tokens.add(token)

    return hindi_tokens, english_tokens

def create_combined_vocabulary(hindi_tokens, english_tokens):
    """Create a combined vocabulary from Hindi and English tokens."""
    combined_vocabulary = hindi_tokens.union(english_tokens)
    return combined_vocabulary

def main(file_path):
    """Main function to load data, split tokens, and print vocabularies."""
    df = load_and_process_data(file_path)

    hindi_tokens, english_tokens = split_tokens_by_language(df, tokenized_text_column='tokenized_text')

    combined_vocabulary = create_combined_vocabulary(hindi_tokens, english_tokens)

    # Print the vocabularies to verify
    print("Hindi Vocabulary:")
    print(len(hindi_tokens))
    print(hindi_tokens)

    print("\nEnglish Vocabulary:")
    print(len(english_tokens))
    print(english_tokens)

    print("\nCombined Vocabulary:")
    print(len(combined_vocabulary))
    print(combined_vocabulary)

if __name__ == "__main__":
    file_path = r'/content/drive/MyDrive/Multiclass_Classification_Internship_NIT_Patna/multiclass_tokenized_cleaned_data.csv'
    main(file_path)

Hindi Vocabulary:
4397
{'खूबियों', 'चूहे', 'दिखेगा', 'कांटे', 'टैक्स', 'किसने', 'नोचना', 'धार्मिक', 'मुफ्त', 'मरेंगे', 'जबरदस्ती', 'सूची', 'लिखने', 'मटेरियल', 'चापलूसों', 'चुस्लाम', 'सार्वजानिकतौर', 'व्यक्ति', 'सर्वोपरि', 'झूठी', 'मां', 'संघी', 'पूजती', 'पंड़ित', 'करनाफैलाना', 'जबरदस्त', 'रैली', 'करवायेंगे', 'भीबिहारी', 'हथियार', 'अजीब', 'सोनिया', 'हैविदेशी', 'राजपाठ', 'विनाश', 'झन्नाटेदार', 'अमृत', 'पापिन', 'गई', 'भगवा', 'मूर्खो', 'सिक्ख', 'फेमस', 'मूत्र', 'चलेगी', 'दादी', 'देहांत', 'चवन्नी', 'बनाया', 'टमाटर', 'सपूत', 'हटाया', 'चमकाने', 'मैगज़ीन', 'उंगलियों', 'निकाला', 'बहूत', 'पाक', 'दियानहेरू', 'दीदी', 'तैयारी', 'किरदारों', 'कुरेदोनहीं', 'भूलेंगे', 'कमाया', 'जब', 'ऐरे', 'सच्ची', 'सेलिब्रिटी', 'हिस्से', 'हिंसक', 'सिर्फ़', 'जाएगीअभी', 'इतिहास', 'इलाकों', 'प्रकाश', 'जातीय', 'बाजे', 'मामला', 'सादगी', 'दलित', 'तडीपार', 'थे', 'हल्के', 'किताब', 'हरामखोर', 'बेवकूफ', 'ट्रांसजेंडर', 'जिदगी', 'चाटो', 'वाही', 'उन्होंने', 'जागो', 'हूं', 'पूँछियेकहाँ', 'उगलते', 'फार्मा', 'सालों', 'कप्तान', 'काशीम

In [34]:
def split_tokens_by_language(df, tokenized_text_column='tokenized_text'):
    hindi_tokens = set()
    english_tokens = set()

    for tokens in df[tokenized_text_column]:
        for token in tokens:
            if re.search('[\u0900-\u097F]', token):  # Check for Hindi characters
                hindi_tokens.add(token)
            else:
                english_tokens.add(token)

    return hindi_tokens, english_tokens

def create_token_id_mapping(tokens):
    return {token: idx for idx, token in enumerate(tokens, start=1)}

def create_combined_token_id_mapping(hindi_tokens, english_tokens):
    combined_tokens = sorted(hindi_tokens.union(english_tokens))
    return {token: idx for idx, token in enumerate(combined_tokens, start=1)}


def unique_ids(file_path):
    # Load the CSV file
    df = pd.read_csv(file_path)

    # Process the dataframe
    df = tokenize_dataframe(df, text_column='cleaned_text')

    # Split tokens into Hindi and English sets
    hindi_tokens, english_tokens = split_tokens_by_language(df, tokenized_text_column='tokenized_text')

    # Create combined token to ID mapping
    combined_token_id_mapping = create_combined_token_id_mapping(hindi_tokens, english_tokens)

    return combined_token_id_mapping, df, hindi_tokens, english_tokens

def convert_tokens_to_ids(df, token_id_mapping):
    # Replace tokens with their corresponding IDs
    df['token_ID'] = df['tokenized_text'].apply(
        lambda tokens: [token_id_mapping.get(token, -1) for token in tokens]
    )
    return df

def separate_tokens_by_language(df, hindi_tokens, english_tokens, token_id_mapping):
    """Separate tokens into English and Hindi with their token IDs."""
    hindi_token_id_mapping = {token: token_id_mapping[token] for token in hindi_tokens if token in token_id_mapping}
    english_token_id_mapping = {token: token_id_mapping[token] for token in english_tokens if token in token_id_mapping}

    return hindi_token_id_mapping, english_token_id_mapping

if __name__ == "__main__":
    file_path = r'/content/drive/MyDrive/Multiclass_Classification_Internship_NIT_Patna/multiclass_tokenized_cleaned_data.csv'

    # Get the token-ID mapping, DataFrame, and token sets
    combined_token_id_mapping, df, hindi_tokens, english_tokens = unique_ids(file_path)

    # Print the token-ID mapping
    print("\nCombined Token to ID Mapping:")
    print(combined_token_id_mapping)

    # Convert tokens to IDs
    df = convert_tokens_to_ids(df, combined_token_id_mapping)

    # Create a new DataFrame with only 'binary_label' and 'token_ID'
    output_df = df[['contextual_label', 'token_ID']]

    # Save the new DataFrame to a CSV file
    output_file_path = r'/content/drive/MyDrive/Multiclass_Classification_Internship_NIT_Patna/multiclass_tokenized_text_as_ids.csv'
    output_df.to_csv(output_file_path, index=False)

    print(f"\nNew CSV file with token IDs saved to: {output_file_path}")

    # Separate English and Hindi tokens with their token IDs
    hindi_token_id_mapping, english_token_id_mapping = separate_tokens_by_language(df, hindi_tokens, english_tokens, combined_token_id_mapping)

    # Print separated token-ID mappings
    print("\nHindi Token to ID Mapping:")
    print(hindi_token_id_mapping)

    print("\nEnglish Token to ID Mapping:")
    print(english_token_id_mapping)

    print(len(hindi_token_id_mapping))
    print(len(english_token_id_mapping))



Combined Token to ID Mapping:
{'aa': 1, 'aaba': 2, 'aabee': 3, 'aabhas': 4, 'aabhi': 5, 'aacha': 6, 'aadhe': 7, 'aadmi': 8, 'aadmiaadmi': 9, 'aaen': 10, 'aaenge': 11, 'aag': 12, 'aagaye': 13, 'aage': 14, 'aagey': 15, 'aagh': 16, 'aaha': 17, 'aai': 18, 'aaise': 19, 'aaj': 20, 'aakankshat': 21, 'aakarpatel': 22, 'aakhri': 23, 'aam': 24, 'aamal': 25, 'aameen': 26, 'aamircapri': 27, 'aamirkhan': 28, 'aamkhe': 29, 'aana': 30, 'aane': 31, 'aankhe': 32, 'aankhein': 33, 'aankho': 34, 'aao': 35, 'aaoge': 36, 'aap': 37, 'aapa': 38, 'aapas': 39, 'aaphonest': 40, 'aapka': 41, 'aapke': 42, 'aapki': 43, 'aapko': 44, 'aapna': 45, 'aapnareshbalyan': 46, 'aapne': 47, 'aapni': 48, 'aapse': 49, 'aarabi': 50, 'aaray': 51, 'aari': 52, 'aarti': 53, 'aartitva': 54, 'aaryaindian': 55, 'aas': 56, 'aashole': 57, 'aasiha': 58, 'aasmani': 59, 'aata': 60, 'aatanbadi': 61, 'aatankbaadi': 62, 'aatanki': 63, 'aatankwadii': 64, 'aatankwadiyo': 65, 'aate': 66, 'aati': 67, 'aatnkwad': 68, 'aatnkwadi': 69, 'aatnkwadiyon

In [8]:
from sklearn.model_selection import train_test_split

# Load the data
file_path = r'/content/drive/MyDrive/Multiclass_Classification_Internship_NIT_Patna/multiclass_tokenized_cleaned_data.csv'
df = pd.read_csv(file_path)

# Define a mapping for the three labels
label_mapping = {'NONE': 0, 'CHOF': 1, 'SHOF': 2}
df['encoded_label'] = df['contextual_label'].map(label_mapping)

# Check for NaN values in cleaned_text
print("Checking for empty entries in 'cleaned_text':")
print(df['cleaned_text'].isna().sum(), "NaN values found.")

# Check if there are any empty entries
print(df[df['cleaned_text'] == ""])

# Features (X) and Labels (y)
X = df['cleaned_text']
y = df['encoded_label']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display split data information
print("Training Set Size:", X_train.shape)
print("Test Set Size:", X_test.shape)

# Load the saved GloVe embeddings
def load_glove_embeddings(filename):
    embeddings = {}
    with open(filename, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            embeddings[word] = vector
    return embeddings

# Load embeddings
embeddings = load_glove_embeddings('/content/drive/MyDrive/Internship_NIT_Patna/mixed_glove_embeddings2.txt')

# Define the sentence_to_embedding function
def sentence_to_embedding(sentence, embeddings, embedding_dim=50):
    words = sentence.split()
    embedding_matrix = np.zeros((len(words), embedding_dim))

    for i, word in enumerate(words):
        if word in embeddings:
            embedding_matrix[i] = embeddings[word]
        else:
            # If the word is not found in the embeddings, use a zero vector
            embedding_matrix[i] = np.zeros(embedding_dim)

    # Aggregate word embeddings into a single sentence embedding, e.g., by averaging
    sentence_embedding = np.mean(embedding_matrix, axis=0)

    return sentence_embedding

# Create embeddings for training and testing data
X_train_embeddings = np.array([sentence_to_embedding(text, embeddings) for text in X_train])
X_test_embeddings = np.array([sentence_to_embedding(text, embeddings) for text in X_test])

# Display embeddings shape
print("Training Embeddings Shape:", X_train_embeddings.shape)
print("Test Embeddings Shape:", X_test_embeddings.shape)

Checking for empty entries in 'cleaned_text':
0 NaN values found.
Empty DataFrame
Columns: [tweet_id, text, contextual_label, cleaned_text, tokenized_text, encoded_label]
Index: []
Training Set Size: (3931,)
Test Set Size: (983,)
Training Embeddings Shape: (3931, 50)
Test Embeddings Shape: (983, 50)


In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the data
file_path = r'/content/drive/MyDrive/Multiclass_Classification_Internship_NIT_Patna/multiclass_tokenized_cleaned_data.csv'
df = pd.read_csv(file_path)

# Encode labels using a LabelEncoder for multiclass classification
label_encoder = LabelEncoder()
df['encoded_label'] = label_encoder.fit_transform(df['contextual_label'])

# Prepare features (X) and labels (y)
X = df['cleaned_text']
y = df['encoded_label']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenize and pad sequences
tokenizer = Tokenizer(num_words=10000)  # Adjust the number of words as needed
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

maxlen = 100  # Maximum sequence length
X_train_padded = pad_sequences(X_train_seq, maxlen=maxlen)
X_test_padded = pad_sequences(X_test_seq, maxlen=maxlen)

# Display the shape of the padded sequences
print("Training Data Shape:", X_train_padded.shape)
print("Testing Data Shape:", X_test_padded.shape)

# Display the label mapping
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Label Mapping:", label_mapping)


Training Data Shape: (3931, 100)
Testing Data Shape: (983, 100)
Label Mapping: {'CHOF': 0, 'NONE': 1, 'SHOF': 2}


In [37]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import ast
import numpy as np

# Logistic Regression Model
logistic_classifier = LogisticRegression()
logistic_classifier.fit(X_train_embeddings, y_train)
y_logistic_pred = logistic_classifier.predict(X_test_embeddings)
logistic_accuracy = accuracy_score(y_test, y_logistic_pred)
logistic_report = classification_report(y_test, y_logistic_pred, target_names=['NONE', 'CHOF', 'SHOF'])
print(f"Logistic Regression Accuracy: {logistic_accuracy * 100:.2f}%")
print("Logistic Regression Classification Report:")
print(logistic_report)
print("\n")

# SVM Model
svm_classifier = SVC()
svm_classifier.fit(X_train_embeddings, y_train)
y_svm_pred = svm_classifier.predict(X_test_embeddings)
svm_accuracy = accuracy_score(y_test, y_svm_pred)
svm_conf_matrix = confusion_matrix(y_test, y_svm_pred)
svm_report = classification_report(y_test, y_svm_pred, target_names=['NONE', 'CHOF', 'SHOF'])
print(f"SVM Accuracy: {svm_accuracy * 100:.2f}%")
print("SVM Confusion Matrix:")
print(svm_conf_matrix)
print("SVM Classification Report:")
print(svm_report)
print("\n")

# Naive Bayes Model
naivebayes_classifier = GaussianNB()
naivebayes_classifier.fit(X_train_embeddings, y_train)
y_naivebayes_pred = naivebayes_classifier.predict(X_test_embeddings)
naivebayes_accuracy = accuracy_score(y_test, y_naivebayes_pred)
naivebayes_conf_matrix = confusion_matrix(y_test, y_naivebayes_pred)
naivebayes_report = classification_report(y_test, y_naivebayes_pred, target_names=['NONE', 'CHOF', 'SHOF'])
print(f"Naive Bayes Accuracy: {naivebayes_accuracy * 100:.2f}%")
print("Naive Bayes Confusion Matrix:")
print(naivebayes_conf_matrix)
print("Naive Bayes Classification Report:")
print(naivebayes_report)
print("\n")

# K-Nearest Neighbors Classifier
knn_classifier = KNeighborsClassifier(n_neighbors=4)
knn_classifier.fit(X_train_embeddings, y_train)
knn_predict = knn_classifier.predict(X_test_embeddings)
knn_accuracy = accuracy_score(y_test, knn_predict)
print("K-Nearest Neighbors Classifier Results:")
print(f"Accuracy: {knn_accuracy * 100:.2f}%")
print("Confusion Matrix:")
print(confusion_matrix(y_test, knn_predict))
print("Classification Report:")
print(classification_report(y_test, knn_predict, target_names=['NONE', 'CHOF', 'SHOF']))
print("\n")

# Decision Tree Classifier
dectree_classifier = DecisionTreeClassifier()
dectree_classifier.fit(X_train_embeddings, y_train)
dectree_predict = dectree_classifier.predict(X_test_embeddings)
dectree_accuracy = accuracy_score(y_test, dectree_predict)
print("Decision Tree Classifier Results:")
print(f"Accuracy: {dectree_accuracy * 100:.2f}%")
print("Confusion Matrix:")
print(confusion_matrix(y_test, dectree_predict))
print("Classification Report:")
print(classification_report(y_test, dectree_predict, target_names=['NONE', 'CHOF', 'SHOF']))
print("\n")

# Gradient Boosting Classifier
gradboost_classifier = GradientBoostingClassifier(random_state=0)
gradboost_classifier.fit(X_train_embeddings, y_train)
gradboost_predict = gradboost_classifier.predict(X_test_embeddings)
gradboost_accuracy = accuracy_score(y_test, gradboost_predict)
print("Gradient Boosting Classifier Results:")
print(f"Accuracy: {gradboost_accuracy * 100:.2f}%")
print("Confusion Matrix:")
print(confusion_matrix(y_test, gradboost_predict))
print("Classification Report:")
print(classification_report(y_test, gradboost_predict, target_names=['NONE', 'CHOF', 'SHOF']))
print("\n")

# Random Forest Classifier
forest_classifier = RandomForestClassifier()
forest_classifier.fit(X_train_embeddings, y_train)
forest_predict = forest_classifier.predict(X_test_embeddings)
forest_accuracy = accuracy_score(y_test, forest_predict)
print("Random Forest Classifier Results:")
print(f"Accuracy: {forest_accuracy * 100:.2f}%")
print("Confusion Matrix:")
print(confusion_matrix(y_test, forest_predict))
print("Classification Report:")
print(classification_report(y_test, forest_predict, target_names=['NONE', 'CHOF', 'SHOF']))
print("\n")


Checking for empty entries in 'cleaned_text':
0 NaN values found.
Empty DataFrame
Columns: [tweet_id, text, contextual_label, cleaned_text, tokenized_text, encoded_label]
Index: []
Training Set Size: (3931,)
Test Set Size: (983,)
Training Embeddings Shape: (3931, 50)
Test Embeddings Shape: (983, 50)
Logistic Regression Accuracy: 55.75%
Logistic Regression Classification Report:
              precision    recall  f1-score   support

        NONE       0.57      0.82      0.67       468
        CHOF       0.64      0.43      0.51       189
        SHOF       0.47      0.26      0.33       326

    accuracy                           0.56       983
   macro avg       0.56      0.50      0.50       983
weighted avg       0.55      0.56      0.53       983



SVM Accuracy: 60.63%
SVM Confusion Matrix:
[[405  12  51]
 [ 63  90  36]
 [207  18 101]]
SVM Classification Report:
              precision    recall  f1-score   support

        NONE       0.60      0.87      0.71       468
        CHO

In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, Embedding, BatchNormalization, LSTM
from tensorflow.keras.regularizers import l2
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
# ------------------ Conv1D Model ------------------

# Define the Conv1D model
model_conv1d = Sequential()
model_conv1d.add(Embedding(input_dim=10000, output_dim=128, input_length=maxlen))
model_conv1d.add(Conv1D(filters=64, kernel_size=5, activation='relu', kernel_regularizer=l2(0.01)))
model_conv1d.add(BatchNormalization())
model_conv1d.add(MaxPooling1D(pool_size=2))
model_conv1d.add(Dropout(0.3))

model_conv1d.add(Conv1D(filters=32, kernel_size=5, activation='relu', kernel_regularizer=l2(0.01)))
model_conv1d.add(BatchNormalization())
model_conv1d.add(MaxPooling1D(pool_size=2))
model_conv1d.add(Dropout(0.3))

model_conv1d.add(Flatten())
model_conv1d.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
model_conv1d.add(Dropout(0.3))
model_conv1d.add(Dense(3, activation='softmax'))  # Change to 3 for multi-class

# Compile the Conv1D model
model_conv1d.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the Conv1D model
history_conv1d = model_conv1d.fit(X_train_padded, y_train, epochs=100, batch_size=32, validation_split=0.3, verbose=2)

# Evaluate the Conv1D model
train_loss_conv1d, train_accuracy_conv1d = model_conv1d.evaluate(X_train_padded, y_train, verbose=2)
print(f"Conv1D Training Loss: {train_loss_conv1d:.4f}")
print(f"Conv1D Training Accuracy: {train_accuracy_conv1d:.4f}")

test_loss_conv1d, test_accuracy_conv1d = model_conv1d.evaluate(X_test_padded, y_test, verbose=2)
print(f"Conv1D Test Loss: {test_loss_conv1d:.4f}")
print(f"Conv1D Test Accuracy: {test_accuracy_conv1d:.4f}")

# Predict with Conv1D model
y_pred_conv1d = np.argmax(model_conv1d.predict(X_test_padded), axis=-1)

# Print classification report for Conv1D model
print("Conv1D Classification Report:")
print(classification_report(y_test, y_pred_conv1d, target_names=['NONE', 'CHOF', 'SHOF']))

# Calculate confusion matrix for Conv1D model
conf_matrix_conv1d = confusion_matrix(y_test, y_pred_conv1d)
print("Conv1D Confusion Matrix:")
print(conf_matrix_conv1d)

Epoch 1/100
86/86 - 8s - 95ms/step - accuracy: 0.4406 - loss: 3.4915 - val_accuracy: 0.4975 - val_loss: 3.0842
Epoch 2/100
86/86 - 5s - 64ms/step - accuracy: 0.5565 - loss: 2.7946 - val_accuracy: 0.4975 - val_loss: 2.6824
Epoch 3/100
86/86 - 4s - 43ms/step - accuracy: 0.6321 - loss: 2.2891 - val_accuracy: 0.4975 - val_loss: 2.3444
Epoch 4/100
86/86 - 4s - 44ms/step - accuracy: 0.7568 - loss: 1.8123 - val_accuracy: 0.5034 - val_loss: 2.0575
Epoch 5/100
86/86 - 7s - 82ms/step - accuracy: 0.8332 - loss: 1.4498 - val_accuracy: 0.5585 - val_loss: 1.8298
Epoch 6/100
86/86 - 8s - 96ms/step - accuracy: 0.8920 - loss: 1.1691 - val_accuracy: 0.5814 - val_loss: 1.6496
Epoch 7/100
86/86 - 5s - 57ms/step - accuracy: 0.9273 - loss: 0.9466 - val_accuracy: 0.5949 - val_loss: 1.5559
Epoch 8/100
86/86 - 4s - 51ms/step - accuracy: 0.9397 - loss: 0.7947 - val_accuracy: 0.5763 - val_loss: 1.5113
Epoch 9/100
86/86 - 4s - 51ms/step - accuracy: 0.9502 - loss: 0.6742 - val_accuracy: 0.5966 - val_loss: 1.6013
E

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


86/86 - 161s - 2s/step - accuracy: 0.4682 - loss: 3.4904 - val_accuracy: 0.4975 - val_loss: 1.4788
Epoch 2/50
86/86 - 201s - 2s/step - accuracy: 0.4835 - loss: 1.2806 - val_accuracy: 0.4975 - val_loss: 1.1598
Epoch 3/50
86/86 - 200s - 2s/step - accuracy: 0.4856 - loss: 1.1218 - val_accuracy: 0.4975 - val_loss: 1.0752
Epoch 4/50
86/86 - 156s - 2s/step - accuracy: 0.4853 - loss: 1.0740 - val_accuracy: 0.4975 - val_loss: 1.0486
Epoch 5/50
86/86 - 202s - 2s/step - accuracy: 0.4853 - loss: 1.0548 - val_accuracy: 0.4975 - val_loss: 1.0336
Epoch 6/50
86/86 - 203s - 2s/step - accuracy: 0.4853 - loss: 1.0416 - val_accuracy: 0.4975 - val_loss: 1.0311
Epoch 7/50
86/86 - 202s - 2s/step - accuracy: 0.4853 - loss: 1.0389 - val_accuracy: 0.4975 - val_loss: 1.0294
Epoch 8/50
86/86 - 203s - 2s/step - accuracy: 0.4853 - loss: 1.0375 - val_accuracy: 0.4975 - val_loss: 1.0272
Epoch 9/50
86/86 - 199s - 2s/step - accuracy: 0.4853 - loss: 1.0330 - val_accuracy: 0.4975 - val_loss: 1.0206
Epoch 10/50
86/86 - 2

In [12]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

# Define the LSTM model
model_lstm = Sequential()
model_lstm.add(Embedding(input_dim=10000, output_dim=128, input_length=maxlen))
model_lstm.add(LSTM(512, return_sequences=True, activation='tanh', kernel_regularizer=l2(0.01)))
model_lstm.add(Dropout(0.2))
model_lstm.add(LSTM(128, activation='tanh', kernel_regularizer=l2(0.01)))
model_lstm.add(Dropout(0.2))
model_lstm.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
model_lstm.add(Dense(3, activation='softmax'))  # Change to 3 for multi-class

# Compile the LSTM model
model_lstm.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Set up callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
# model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss', mode='min')

# Train the LSTM model with callbacks
history_lstm = model_lstm.fit(X_train_padded, y_train, epochs=50, batch_size=32, validation_split=0.3,
                              callbacks=[early_stopping], verbose=2)

# Evaluate the LSTM model
train_loss_lstm, train_accuracy_lstm = model_lstm.evaluate(X_train_padded, y_train, verbose=2)
print(f"LSTM Training Loss: {train_loss_lstm:.4f}")
print(f"LSTM Training Accuracy: {train_accuracy_lstm:.4f}")

test_loss_lstm, test_accuracy_lstm = model_lstm.evaluate(X_test_padded, y_test, verbose=2)
print(f"LSTM Test Loss: {test_loss_lstm:.4f}")
print(f"LSTM Test Accuracy: {test_accuracy_lstm:.4f}")

# Predict with LSTM model
y_pred_lstm = np.argmax(model_lstm.predict(X_test_padded), axis=-1)

# Print classification report for LSTM model
print("LSTM Classification Report:")
print(classification_report(y_test, y_pred_lstm, target_names=['NONE', 'CHOF', 'SHOF']))

# Calculate confusion matrix for LSTM model
conf_matrix_lstm = confusion_matrix(y_test, y_pred_lstm)
print("LSTM Confusion Matrix:")
print(conf_matrix_lstm)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/50
86/86 - 184s - 2s/step - accuracy: 0.4918 - loss: 3.4006 - val_accuracy: 0.5568 - val_loss: 1.3448
Epoch 2/50
86/86 - 186s - 2s/step - accuracy: 0.6034 - loss: 1.0825 - val_accuracy: 0.5941 - val_loss: 1.0178
Epoch 3/50
86/86 - 201s - 2s/step - accuracy: 0.7263 - loss: 0.7943 - val_accuracy: 0.6110 - val_loss: 0.9286
Epoch 4/50
86/86 - 154s - 2s/step - accuracy: 0.8110 - loss: 0.6141 - val_accuracy: 0.6068 - val_loss: 0.9779
Epoch 5/50
86/86 - 203s - 2s/step - accuracy: 0.8481 - loss: 0.4948 - val_accuracy: 0.5983 - val_loss: 1.0975
Epoch 6/50
86/86 - 202s - 2s/step - accuracy: 0.8760 - loss: 0.4286 - val_accuracy: 0.5975 - val_loss: 1.2723
Epoch 7/50
86/86 - 162s - 2s/step - accuracy: 0.8920 - loss: 0.3924 - val_accuracy: 0.5763 - val_loss: 1.2022
Epoch 8/50
86/86 - 155s - 2s/step - accuracy: 0.9029 - loss: 0.3567 - val_accuracy: 0.5746 - val_loss: 1.2176
Epoch 9/50
86/86 - 162s - 2s/step - accuracy: 0.9117 - loss: 0.3182 - val_accuracy: 0.5839 - val_loss: 1.4639
Epoch 10/5